In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
# pip install openpyxl

In [3]:
nswmap = pd.read_excel('../data/nsw-site-location-and-details.xlsx')
print(nswmap.shape)
nswmap.head(3)

(2255, 12)


,Site2,Site_distance,Community,Latitude,Longitude,Easting,Northing,Zone,Datum,Location name,Tenure,Date sampled
0,CYP_001,CYP_001_000,Cypress pine,-32.314901,145.820500,6424048,388969,55,MGA94,Bedooba State Conservation Area,National Park,2013-09-11
1,CYP_001,CYP_001_050,Cypress pine,-32.314476,145.820346,6424095,388954,55,MGA94,Bedooba State Conservation Area,National Park,2013-09-11
2,CYP_001,CYP_001_100,Cypress pine,-32.314051,145.820182,6424142,388938,55,MGA94,Bedooba State Conservation Area,National Park,2013-09-11


In [5]:
soil = pd.read_excel('../data/nsw-assessment-of-soil-health.xlsx')
print(soil.shape)
soil.head(3)

(2255, 26)


,Community,SiteID,Site,Rough,Resist,Resist_adjust,Broke,Stab,Biocrust,Biocrust_code,...,Litter_code,Litt_depth,Litt_Origin,Litt_Incorp,Litt_Propn,Litt_Complex,Texture,Stability,Infiltration,Nutrients
0,Cypress pine,CYP_001_000,CYP_001,1,4,1.0,2,4,5.0,2,...,1,3.0,1.5,1.0,95.0,1.5,2,54.107231,20.550760,15.116279
1,Cypress pine,CYP_001_050,CYP_001,1,4,1.0,4,4,20.0,3,...,2,4.0,1.5,1.0,30.0,3.0,2,63.944909,22.605836,20.930233
2,Cypress pine,CYP_001_100,CYP_001,1,4,1.0,4,4,10.0,2,...,2,0.5,1.5,1.0,100.0,3.0,2,63.944909,20.550760,16.279070


In [ ]:
nswmap.info()

In [ ]:
nswmap['geometry'] = nswmap.apply(lambda x: Point((x.Longitude, 
                                                         x.Latitude)), 
                                        axis=1)
nswmap.head(3)

In [ ]:
nswmap = gpd.GeoDataFrame(nswmap, 
                           crs = 'EPSG:4326', 
                           geometry = nswmap['geometry'])

In [ ]:
nswmap.crs

In [ ]:
type(nswmap)

In [ ]:
nswmap.plot()

In [ ]:
geodata = gpd.read_file('../data/STE_2021_AUST_GDA2020.shp')
nsw = geodata[geodata['STE_NAME21']== 'New South Wales']
nsw.plot()

In [ ]:
nsw.crs = nswmap.crs

In [ ]:
nsw.crs

In [ ]:
nsw.plot()

In [ ]:
nswmap.head()

In [ ]:
type(nsw)

In [ ]:
nsw_map = gpd.sjoin(nsw, nswmap, predicate = 'within')

In [ ]:
border = nsw.loc[nsw['STE_NAME21'] == 'New South Wales']
border.shape

In [ ]:
ax = border.plot(figsize = (8, 10), color = 'lightgreen')
nswmap.plot( ax = ax, column = 'Community');
plt.show();

In [ ]:
border.geometry.centroid

In [ ]:
center = border.geometry.centroid[0]

In [ ]:
print(center)

## Folium requires a location point as an array with latitude first
but shapely Points have longitude first

In [ ]:
center.y

In [ ]:
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

# check the order of area_center, the location for our folium map
print(area_center)

To create a folium map just call the folium Map() constructor
location is the only argument required by the constructor
Set area_center as the location of the folium map.
zoom_start is an optional argument
A setting of 12 should get us nice and close

In [ ]:
#create our map of NSW and show it
map_nsw = folium.Map(location =  area_center, zoom_start = 10)
map_nsw

#### Next, let's add our collections points


In [ ]:
nswmap.head(1)

In [ ]:
#draw our zip code area: 37207
map_nsw = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(border).add_to(map_nsw)

#iterate through stops_in_37207 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in nswmap.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Location name'])
    icon=folium.Icon(color="blue",icon="binoculars", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, 
        icon = icon) 

    marker.add_to(map_nsw)
# map_37207.save('../maps/map37207.html')

#display our map
map_nsw